# Data Mining II Project 
***
#### University of Pisa - Gabriele Leone, Federica Trevisan

# Time series
Given the 50+ years long history of stock values of a company, split it into years, and study their similarities, also using clustering.

## Dataset
IBM stocks (source: Yahoo Finance). Dataset obtained from Yahoo!Finance service.

## Objectives
Compare similarities, compute clustering.